Prerequisite: create an index with 1536 dimensions - in case you also use OpenAI's 'text-embedding-ada-002' for embedding

In [1]:
import os
import openai
import json

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY") or "OPENAI_API_KEY"
# openai.Engine.list()

In [3]:
cnvrg = []
with open("cnvrg_docs.json", "r", encoding="utf-8") as input_file:
    input_json = json.load(input_file)
    cnvrg = input_json["messages"]

In [4]:
len(cnvrg)

486

In [65]:
qna = []
switchup = 0
for qa in cnvrg:
    q = qa["content"].strip()
    a = qa["response"].strip()
    if not q or not a:
        continue
    if (a.startswith("How ") or a.startswith("What ") or a.startswith("Who ") or a.startswith("Can ") or a.startswith("Where ") or a.startswith("Are ")) and not (
        q.startswith("How ") or q.startswith("What ") or q.startswith("Who ") or q.startswith("Can ") or q.startswith("Where ") or q.startswith("Are ")
    ):
        q, a = a, q
        switchup += 1
    qna.append((q, a))

In [66]:
switchup

431

In [22]:
embed_model = "text-embedding-ada-002"

In [72]:
qs = [q for q, a in qna]

In [73]:
res = openai.Embedding.create(input=qs, engine=embed_model)

In [74]:
res.keys()  # Executes the lazy query

dict_keys(['object', 'data', 'model', 'usage'])

In [75]:
len(res['data'])

464

In [41]:
!pip install -qU "pinecone-client[grpc]"==2.2.1 tqdm

In [42]:
import tqdm
import pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.getenv("PINECONE_API_KEY") or "PINECONE_API_KEY"
# find your environment next to the api key in pinecone console
env = os.getenv("PINECONE_ENVIRONMENT") or "gcp-starter"

In [43]:
pinecone.init(api_key=api_key, environment=env)
pinecone.whoami()

WhoAmIResponse(username=None, user_label=None, projectname='14d2815')

In [44]:
index_name = 'cnvrg-openai'
# connect to index
index = pinecone.GRPCIndex(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}

In [76]:
to_upsert = []
for idx, qa in enumerate(qna):
    to_upsert.append((str(idx), res['data'][idx]['embedding'], dict(question=qa[0], answer=qa[1])))

In [77]:
idx = 0
upload_batch_size = 20
# 2MB max request size!
while idx < len(to_upsert):
    index.upsert(vectors=to_upsert[idx:idx+upload_batch_size])
    idx += upload_batch_size

In [81]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.00464,
 'namespaces': {'': {'vector_count': 464}},
 'total_vector_count': 464}

In [82]:
res = openai.Embedding.create(
    input=["Visual Studio Code"],
    engine=embed_model
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(xq, top_k=5, include_metadata=True)

In [ ]:
res

In [71]:
# index.delete(ids=[str(i) for i in range(len(qna))])